In [ ]:
import pyansys
from pyansys import examples

In [ ]:
filename = examples.hexarchivefile

In [ ]:
archive = pyansys.Archive(filename)

In [ ]:
for key in archive.raw:
     print("%s : %s" % (key, archive.raw[key]))

In [ ]:
# Create a vtk unstructured grid from the raw data and plot it
grid = archive.parse_vtk(force_linear=True)
grid.plot(color='w', show_edges=True)

In [ ]:
import pyansys
from pyansys import examples

# load example beam result file
result = pyansys.read_binary(examples.rstfile)

# save as a binary vtk xml file
result.save_as_vtk('beam.vtu')

#### output pressure data for ECR-calcution
- select relevant nodes
- write the require paremeters in ansys apdl
- write out the pressure data  with time  and save as csv

In [ ]:
ansys.prep7()
ansys.allsel('all')
ansys.upgeom(1, 3, 'last', STRUCT_FILE, 'rst')

In [ ]:
ansys.post1()
ansys.lsel('s', '', '', 200, 202, 1)
ansys.nsll('s', 1)
ansys.esln('s', 0)
ansys.nsle('r', 'corner')
ansys.nplot()
ansys.load_parameters()
ansys.parameters

#### write the require paremeters in ansys apdl

In [ ]:
# define the name of the substeps 
ansys.post1()
par = ['NMBR_SS_PF', 'NMBR_SS_SP', 'NMBR_SS_SWF', 'NMBR_SS_EI']
ansys.run("ND_PRES_CUR = 0")
ansys.load_parameters()

# get the substeps number of eath Loadstep
for i in range(len(par)):
    ansys.set(i + 1, 'last')
    ansys.get(par[i], 'active', 0, 'solu', 'ncmss')
    ansys.load_parameters()

NMBR_SS = sum([ansys.parameters[ele] for ele in par])
NMBR_SS_EI = ansys.parameters['NMBR_SS_EI']

# count the number of node on contact surface
ansys.get('CNTCT1_ND_NMR', 'node', 0, 'count')
# get the max nodenumber of the selected nodes
ansys.get('CNTCT1_ND_MAX', 'node', 0, 'num', 'max')
# get the min nodenumber of the selected nodes
ansys.get('CNTCT1_ND_MIN', 'node', 0, 'num', 'min')
# build a node time press tab
ansys.dim('CNTCT1_ND_PRES', 'table', 'CNTCT1_ND_NMR', NMBR_SS_EI, '', '', 'time')
ansys.load_parameters()
ansys.parameters

####  write out the pressur data e with time and save as csv 
- durch for loop werden
    - min node Nummer == aktelle node Number
    - zuerst die Zeit von einem Substep gekregt
    - schreiben die zeit als columns von Tab
    - kommen zu einer anderen Loop
        - schreiben die aktuelle node Number ald index von Tab
        - kriegen Pressure von akutelle node Number
        - kriegen die nächste node Number
- Schreibe aus als csv Datei
- Daten erkennen
- TODO: change name of dataframe

In [ ]:
ansys.set(3,'last')
df = pd.DateFrame()

In [ ]:
for i in range(int(NMBR_SS_EI)):
    CNTCT1_ND_CUR = CNTCT1_ND_MIN
    ansys.get('TIME_LS', 'active', 0, 'set', 'time')
    ansys.load_parameters()
    time = ansys.parameters['TIME_LS']
    time_list.append(time)
    for j in range(int(CNTCT1_ND_NMR)):
        if i == 0:
            ND_list.append(CNTCT1_ND_CUR)

        ansys.get('ND_PRES_CUR', 'node', CNTCT1_ND_CUR, 'cont', 'pres')
        ansys.load_parameters()
        pres = ansys.parameters['ND_PRES_CUR']
        pres_list.append(pres)
        ansys.get('CNTCT1_ND_CUR', 'node', CNTCT1_ND_CUR, 'nxth')
        ansys.load_parameters()
        CNTCT1_ND_CUR = ansys.parameters['CNTCT1_ND_CUR']
        ansys.run("ND_PRES_CUR = 0")

    df[time] = pres_list
    pres_list.clear()
    ansys.set('next')

#### Test Berechnungszeit optimieren

In [ ]:
# Importiere benötigte Bibliotheken
import os
import pyansys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
from IPython.display import Image
from scipy.interpolate import interp1d
import time as zeit

In [ ]:
# cwd change
def Newdir(path):
    import os
    # new dir
    try:
        os.makedirs(path)
    except FileExistsError:
        print (path + '\t directory already exists!')
    # cwd
    os.chdir(path)
    print('cwd:', os.getcwd())

subfolder = 'Output'

In [ ]:
def SaveAndExit():
    ansys.save()
    ansys.Exit()

In [ ]:
# ansys apdl aktivieren
filecode = 'therm_cdw_ring_al_py'           # TODO
cwd  = 'E:/Lu/SHK/Pyansys/Solution/'                     # TODO
mkpath = cwd + filecode
Newdir(mkpath)
# physiscs name
PHYS_STRUCT       = 'Struct'
PHYS_T_E          = 'T_E'
STRUCT_FILE       = 'Structure'
T_E_FILE          = 'Thermo_Electric'
PRMTR_DT = 'PRMTR_DT'
ansys = pyansys.Mapdl(run_location = os.getcwd(),jobname = STRUCT_FILE, interactive_plotting = True)
ansys.resume('{}/{}'.format(mkpath,STRUCT_FILE),'db')

In [ ]:
ansys.prep7()
ansys.allsel('all')
ansys.upgeom(1, 3, 'last', STRUCT_FILE, 'rst')

In [ ]:
ansys.post1()
ansys.lsel('s', '', '', 200, 202, 1)
ansys.nsll('s', 1)
ansys.esln('s', 0)
ansys.nsle('r', 'corner')
ansys.nplot()
ansys.load_parameters()
ansys.parameters

In [ ]:
Image(filename='{}/{}006.png'.format(mkpath,STRUCT_FILE),height=600,width = 600)

In [ ]:
# define the name of the substeps 
ansys.post1()
par = ['NMBR_SS_PF', 'NMBR_SS_SP', 'NMBR_SS_SWF', 'NMBR_SS_EI']
ansys.run("ND_PRES_CUR = 0")
ansys.load_parameters()

# get the substeps number of eath Loadstep
for i in range(len(par)):
    ansys.set(i + 1, 'last')
    ansys.get(par[i], 'active', 0, 'solu', 'ncmss')
    ansys.load_parameters()

NMBR_SS = sum([ansys.parameters[ele] for ele in par])
NMBR_SS_EI = ansys.parameters['NMBR_SS_EI']

# count the number of node on contact surface
ansys.get('CNTCT1_ND_NMR', 'node', 0, 'count')
# get the max nodenumber of the selected nodes
ansys.get('CNTCT1_ND_MAX', 'node', 0, 'num', 'max')
# get the min nodenumber of the selected nodes
ansys.get('CNTCT1_ND_MIN', 'node', 0, 'num', 'min')
# build a node time press tab
CNTCT1_ND_PRES = 'CNTCT1_ND_PRES'
ansys.dim(CNTCT1_ND_PRES, 'table', 'CNTCT1_ND_NMR', NMBR_SS_EI, '', '', 'time')
ansys.load_parameters()
ansys.parameters

In [ ]:
time_col = []
node_index = []
ansys.set(3,'last')
ansys.uis('msgpop',3)
df2 = pd.DataFrame()
CNTCT1_ND_MIN = ansys.parameters['CNTCT1_ND_MIN']
CNTCT1_ND_NMR = ansys.parameters['CNTCT1_ND_NMR']
NMBR_SS_EI = ansys.parameters['NMBR_SS_EI']

for i in range(int(NMBR_SS_EI)):
    CNTCT1_ND_CUR = CNTCT1_ND_MIN
    ansys.get('TIME_LS', 'active', 0, 'set', 'time')
    ansys.load_parameters()
    TIME_LS = ansys.parameters['TIME_LS']
    time_col.append(TIME_LS)
    ansys.taxis('CNTCT1_ND_PRES(1,{})'.format(i+1),2,TIME_LS)
    for j in range(int(CNTCT1_ND_NMR)):    
        if i == 0:
            ansys.taxis('CNTCT1_ND_PRES({},1)'.format(j+1),1,CNTCT1_ND_CUR)
        ansys.get('CNTCT1_ND_PRES({},{})'.format(j+1,i+1),'node',CNTCT1_ND_CUR,'cont', 'pres')
        ansys.get('CNTCT1_ND_CUR', 'node', CNTCT1_ND_CUR, 'nxth')
        ansys.load_parameters()
        CNTCT1_ND_CUR = ansys.parameters['CNTCT1_ND_CUR']
        node_index.append(CNTCT1_ND_CUR)
        
    ansys.set('next')

In [ ]:
ansys.allsel('all')
ansys.prep7()
ansys.upgeom(-1, 3, 'last', STRUCT_FILE, 'rst')
ansys.allsel('all')
ansys.parsav('all', 'PRMTR_DT', 'sav')
ansys.save()
ansys.finish()

In [ ]:
SaveAndExit()